**Today Topics**

```
Working with APIs
SQLAlchemy
```

**Working with Web & APIs**

Better to use the request for fetching data, instead of urllib

In [7]:
from urllib.request import urlopen

In [8]:
url = "https://anandology.com/tmp/hello.txt"

In [9]:
urlopen(url).read()

b'Hello, world!\n'

In [10]:
urlopen(url).read().decode('utf-8')

'Hello, world!\n'

In [12]:
response = urlopen(url)

In [13]:
response

In [14]:
print(response.status)

200


In [15]:
print(response.headers)

Server: nginx/1.10.3 (Ubuntu)
Date: Fri, 11 Feb 2022 08:55:37 GMT
Content-Type: text/plain
Content-Length: 14
Last-Modified: Sat, 30 Nov 2019 10:26:32 GMT
Connection: close
ETag: "5de243d8-e"
Access-Control-Allow-Origin: *
Accept-Ranges: bytes




In [16]:
!curl https://anandology.com/tmp/hello.txt

Hello, world!


In [17]:
!curl -i https://anandology.com/tmp/hello.txt

HTTP/1.1 200 OK
Server: nginx/1.10.3 (Ubuntu)
Date: Fri, 11 Feb 2022 08:56:50 GMT
Content-Type: text/plain
Content-Length: 14
Last-Modified: Sat, 30 Nov 2019 10:26:32 GMT
Connection: keep-alive
ETag: "5de243d8-e"
Access-Control-Allow-Origin: *
Accept-Ranges: bytes

Hello, world!


The `requests` Module

There is a very popular third-party library called requests for handling HTTP requests.

To install it, use:

In [19]:
import requests

In [20]:
url = "https://anandology.com/tmp/hello.txt"

In [21]:
response = requests.get(url)

In [22]:
response.status_code

200

In [23]:
response.headers

{'Server': 'nginx/1.10.3 (Ubuntu)', 'Date': 'Fri, 11 Feb 2022 08:58:10 GMT', 'Content-Type': 'text/plain', 'Content-Length': '14', 'Last-Modified': 'Sat, 30 Nov 2019 10:26:32 GMT', 'Connection': 'keep-alive', 'ETag': '"5de243d8-e"', 'Access-Control-Allow-Origin': '*', 'Accept-Ranges': 'bytes'}

In [24]:
# read the response as bytes
response.content

b'Hello, world!\n'

In [25]:
# read the response as text
response.text 

'Hello, world!\n'

**Problem:**
Write a program sumfile.py that takes a filename as command-line argument and prints sum of all numbers in that file. It is expected that the file will have one number for every line. The program should ignore the line if it is not a valid number after printing a warning message. The warning message should be printed to sys.stderr.
```
$ python sumfile.py files/numbers.txt
WARNING: Bad number 'N/A'
WARNING: Bad number 'xxx'
15
```

In [79]:
%%file numbers.txt
1
2
3
N/A
4
xxx
5


Overwriting numbers.txt


In [94]:
%%file sumfile.py
import sys
filename = sys.argv[1]
with open(filename,'r') as f:
    sum = 0
    a = f.readlines()
    for i in a:
        try:
            sum+=int(i.strip('\n'))
        except ValueError:
            sys.stderr.write("Warning: Bad Number " + i)
    print(sum)

Overwriting sumfile.py


In [95]:
!python sumfile.py numbers.txt

15


In [132]:
from sqlalchemy import create_engine
from sqlalchemy import MetaData, Table
from sqlalchemy import desc
from sqlalchemy import select

In [133]:
engine = create_engine("sqlite:///trains.db")

In [134]:
metadata = MetaData(bind=engine)

In [135]:
train_table = Table('train', metadata, autoload=True)
station_table = Table('station', metadata, autoload=True)

In [136]:
for c in train_table.columns:
    print(c)

train.number
train.name
train.type
train.zone
train.from_station_code
train.from_station_name
train.to_station_code
train.to_station_name
train.departure
train.arrival
train.duration_h
train.duration_m
train.distance
train.return_train
train.sleeper
train.third_ac
train.second_ac
train.first_ac
train.first_class
train.chair_car


In [137]:
for c in station_table.columns:
    print(c)

station.code
station.name
station.zone
station.state
station.address
station.latitude
station.longitude


In [151]:
%%file find-trains.py
import sys
from sqlalchemy import create_engine
from sqlalchemy import MetaData, Table
from sqlalchemy import desc
from sqlalchemy import select

engine = create_engine("sqlite:///trains.db")
metadata = MetaData(bind=engine)
train_table = Table('train', metadata, autoload=True)
station_table = Table('station', metadata, autoload=True)

t = train_table
s = station_table

q = (select(
        t.c.departure,
        t.c.arrival,
        t.c.number,
        t.c.name,
        t.c.distance)
     .where(s.c.code==sys.argv[2], s.c.code==sys.argv[2] )
     .order_by(t.c.departure.desc())
     .limit(10))   
print(q)

Overwriting find-trains.py


In [152]:
!python find-trains.py SBC MAS

SELECT train.departure, train.arrival, train.number, train.name, train.distance 
FROM train, station 
WHERE station.code = ? AND station.code = ? ORDER BY train.departure DESC
 LIMIT ? OFFSET ?


In [154]:
result = q.execute()
for row in result:
    print(row)

('None', 'None', '24041', 'DELHI SARAI ROHILLA - KOTDWAR Mussoorie Exp', '')
('None', 'None', '12374', 'RAMPURHAT - SEALDAH Intercity SF Exp', '')
('None', 'None', '12373', 'SEALDAH - RAMPURHAT Intercity SF Exp', '')
('None', 'None', '13258', 'ANAND VIHAR DELHI - DANAPUR JanSadharan Exp', '')
('None', 'None', '13257', 'DANAPUR - DELHI ANAND VIHAR Jan Sadharan Express', '')
('None', 'None', '57326', 'GUNTUR - KACHEGUDA PASS', '')
('None', 'None', '57325', 'KACHEGUDA - GUNTUR PASS', '')
('None', 'None', '12163', 'DADAR - CHENNAI EGMORE SF Exp', '')
('None', 'None', '12164', 'DADAR - CHENNAI EGMORE SF Exp', '')
('None', 'None', '12735', 'SECUNDERABAD - YESVANTPUR Garib Rath Exp', '')


In [153]:
import sqlalchemy
sqlalchemy.__version__

'1.4.31'

In [131]:
t = train_table

q = (select(
        t.c.number,
        t.c.name,
        t.c.from_station_code,
        t.c.to_station_code,
        t.c.distance)
     .where(t.c.distance != '')
     .order_by(t.c.distance.desc())
     .limit(10))     
print(q)

NameError: name 'select' is not defined